# Find hospitals closest to an incident

This jupyter notebook constitutes the Exercise Sheet for the first lecture aput the arcgis API for Python.

It is build as a 'gap text' challenge, there is a lot of text, code snippets and comments already added. To complete this exercise follow the instructions to build a fully working jupyter notebook. Add comments if necessary. 

You will use the functions of the network analyst module from the arcgis api for Python to build a solver the finds the route and the distance in kilometers towards the next hospital in Münster. 

There are two bonus challenges for 2 points each:
1. Build a Interactive Input for the Incident Coordinates
2. Label the result of the routing with the kilometer distance

You will find the details for the bonus challenge in the explanations.

There are 10 Points for the exercise and 4 bonus points available.
Please remember to write code that is easy readable, well documented and actually working. 

The `network` module of the ArcGIS API for Python can be used to solve different types of network analysis operations. In this Exercise, we see how to find the hospital that is closest to an incident.

## Closest facility

The closest facility solver provides functionality for finding out the closest locations to a particular input point. This solver would be useful in cases when you have an incident and need to find the closest facility or need to get information on the travel time and the distance to each of the facilities from an incident point for reporting purposes.

![](http://desktop.arcgis.com/en/arcmap/latest/extensions/network-analyst/GUID-96C273DB-6A24-4D42-AADA-975A33B44F3D-web.png)

When finding closest facilities, you can specify how many to find and whether the direction of travel is toward or away from them. The closest facility solver displays the best routes between incidents and facilities, reports their travel costs, and returns driving directions.

### Connect to your GIS
As a first step, you would need to establish a connection to your organization which could be an ArcGIS Online organization or an ArcGIS Enterprise.

In [22]:
from IPython.display import HTML
import pandas as pd
from arcgis.gis import GIS
import getpass

# Connect to your GIS 
username = input("Enter your username: ") # I don't know why getpass.getuser doesn't work here
password = getpass.getpass("Enter your password: ")

my_gis = GIS(url="https://www.arcgis.com", username=username, password=password)

# I followed this example:
# https://developers.arcgis.com/python/guide/create-data/



### Create a Network Layer
To perform any network analysis (such as finding the closest facility, the best route between multiple stops, or service area around a facility), you would need to create a `NetworkLayer` object. In this sample, since we are solving for closest facilities, we need to create a `ClosestFacilityLayer` which is a type of `NetworkLayer`.

To create any `NetworkLayer` object, you would need to provide the URL to the appropriate network analysis service. Hence, in this sample, we provide a `ClosestFacility` URL to create a `ClosestFacilityLayer` object. 

Since all ArcGIS Online organizations already have access to those routing services, you can access this URL through the `GIS` object's `helperServices` property. If you have your own ArcGIS Server based map service with network analysis capability enabled, you would need to provide the URL for this service.

Let us start by importing the `network` module

In [23]:
import arcgis.network as network

Access the analysis URL from the `GIS` object

In [24]:
analysis_url = my_gis.properties.helperServices.closestFacility.url
analysis_url

'https://route.arcgis.com/arcgis/rest/services/World/ClosestFacility/NAServer/ClosestFacility_World'

Create a `ClosestFacilityLayer` object using this URL

See the help for details in Syntax. https://developers.arcgis.com/python/api-reference/arcgis.network.toc.html#closestfacilitylayer

In [25]:
cf_layer = network.ClosestFacilityLayer(analysis_url, gis=my_gis)

### Create hospitals layer
In this sample, we will be looking for the closest hospital (facility) to an incident location. Even though we are interested in finding out the closest one, it would still be helpful to get the information on the distance and travel time to all of them for reference purposes.

In the code below, we need to geocode the hospitals' addresses as well as do the reverse geocode for the incident location which has been supplied in the latitude/longitude format.

To perform the geocode operations, we import the `geocoding` module of the ArcGIS API.

In [26]:
from arcgis import geocoding

In this sample, we geocode addresses of hospitals to create the facility layer. In your workflows, this could any feature layer. Create a list of hospitals in Münster, Germany

In [27]:
hospitals_addresses = ['Dorbaumstraße 300, 48157 Münster, Germany',
                       'Alexianerweg 9, 48163 Münster, Germany',
                       'Westfalenstr. 109, 48165 Münster, Germany',
                       'Loerstraße 23, 48143 Münster, Germany',
                       'Hohenzollernring 70, 48145 Münster, Germany',
                       'Wichernstraße 8, 48147 Münster, Germany',
                       'Domagkstr. 5, 48149 Münster, Germany']

Loop through each address and geocode it. The geocode operation returns a list of matches for each address. We pick the first result and extract the coordinates from it and construct a `Feature` object out of it. Then we combine all the `Feature`s representing the hospitals into a `FeatureSet` object.

Geocoding syntax: https://developers.arcgis.com/python/api-reference/arcgis.geocoding.html#geocode

Feature Syntax: https://developers.arcgis.com/python/api-reference/arcgis.features.toc.html#feature

Feature Set Syntax: https://developers.arcgis.com/python/api-reference/arcgis.features.toc.html#featureset

Hint: Check the structure from the return element of the geocoder to properly construct your feature. Use a loop to run through the list of adresses, add each result feature to a python list and use this python list to build your feature set. While building the feature set, use this as the value for the spatial_reference parameter `{'wkid' : 4326, 'latestWkid': 4326}`

In [28]:
from arcgis.features import Feature, FeatureSet

In [29]:
hosp_feat_list = []

for address in hospitals_addresses:
    geocode_result = geocoding.geocode(address)[0]  # Get the first match
    location = geocode_result['location']
    
    # Create a Feature object
    feature = Feature(geometry={'x': location['x'], 'y': location['y'], 'spatialReference': {'wkid': 4326}})
    hosp_feat_list.append(feature)

Construct a `FeatureSet` using each hospital `Feature`.

In [30]:
hospitals_fset = FeatureSet(features=hosp_feat_list, spatial_reference={'wkid': 4326, 'latestWkid': 4326})

Lets draw our hospitals on a map. To do so, intanciate a gis.map object, focused in Münster. Set the basemap to `arcgis-light-gray`

Details about the map widget can be found here https://developers.arcgis.com/python/api-reference/arcgis.gis.toc.html#arcgis.gis.GIS.map
use the map.draw() function to add the hospitals to the map. You can use the following value for the 'symbol' parameter: `{"type": "esriSMS","style": "esriSMSSquare","color": [76,115,0,255],"size": 8,}`

In [31]:
from IPython.display import HTML


In [32]:
# instanciate the map
map1 = my_gis.map("Münster, Germany")
map1.basemap = 'arcgis-light-gray'

# symbology for the hospitals
symbol = {
    "type": "esriSMS",
    "style": "esriSMSSquare",
    "color": [76,115,0,255],
    "size": 8,
}

In [33]:
# draw the hospitals on the map
map1.draw(hospitals_fset, symbol = symbol)
map1

MapView(layout=Layout(height='400px', width='100%'))

### Create incidents layer
Lets use the geocoding.reverse_geocoding function to build an Feature set from the provided incident coordinates. Feel free to use your own coordinates or use the one provided

The syntax for the reverse geocoding is here: https://developers.arcgis.com/python/api-reference/arcgis.geocoding.html#reverse-geocode

1. Reverse Geocode the coordinates
2. build a arcgis.Feature from the result object, using the 'location' from the response object as a parameter for the `geometry` parameter and the 'adress' from the response object for the `attributes` parameter of the Feature.
3. Build a Feature set from the one Feature, similar to the feature set for the hospitals. You can use the same spatial_reference parameter

## Bonus Challenge: Build an interactive Input for the coordinates
for 2 extra points: 
Open an input form and verify that the input string has the format of XX.XXXX, YY.YYYY

Give Feedback to the user if the format is correct or not.

In [50]:
import re

# Open an input form and verify that the input string has the format of XX.XXXX, YY.YYYY
coords = input("Enter the coordinates OR type in 'no' to use default coords: ")
if re.match("^-?\\d+(\\.\\d+)?,\\s*-?\\d+(\\.\\d+)?$", coords):
    print("The format is correct.")  # Give Feedback to the user if the format is correct or not.
elif coords == "no":  # If the user enters "no", use the default coordinates
    coords = "7.641378810646356,51.95304243253929"
    print("You decided to use the default coordinates.")
else:
    print("The format is incorrect. Please try again.")  # Give Feedback to the user if the format is correct or not.


You decided to use the default coordinates.


In [42]:
# reverse geocode & build feature
incident_coords = coords
result = geocoding.reverse_geocode([float(x) for x in incident_coords.split(',')])

In [43]:
# build the incident feature set
incident_feature = Feature(geometry={'x': result['location']['x'], 
                                     'y': result['location']['y'], 
                                     'spatialReference': {'wkid': 4326}},
                           attributes=result['address'])

incident_fset = FeatureSet(features=[incident_feature], spatial_reference={'wkid': 4326, 'latestWkid': 4326})

In [37]:
# check for the result (this cell does not need to be changed)
incident_fset.features

[{"geometry": {"x": 7.641618030163, "y": 51.953028183304, "spatialReference": {"wkid": 4326}}, "attributes": {"Match_addr": "Dortmunder Stra\u00dfe 34, 48155, M\u00fcnster, Mitte-Nordost, Mauritz, Nordrhein-Westfalen", "LongLabel": "Dortmunder Stra\u00dfe 34, 48155, M\u00fcnster, Mitte-Nordost, Mauritz, Nordrhein-Westfalen, DEU", "ShortLabel": "Dortmunder Stra\u00dfe 34", "Addr_type": "PointAddress", "Type": "", "PlaceName": "", "AddNum": "34", "Address": "Dortmunder Stra\u00dfe 34", "Block": "", "Sector": "", "Neighborhood": "Mauritz", "District": "Mitte-Nordost", "City": "M\u00fcnster", "MetroArea": "", "Subregion": "M\u00fcnster", "Region": "Nordrhein-Westfalen", "RegionAbbr": "NW", "Territory": "", "Postal": "48155", "PostalExt": "", "CntryName": "Deutschland", "CountryCode": "DEU", "X": 7.641618030163, "Y": 51.953028183304, "InputX": 7.641378810646, "InputY": 51.953042432539, "OBJECTID": 1}}]

Draw the incident Feature Set on the map, using this value for the 'symbol' parameter `{"type": "esriSMS","style": "esriSMSCircle","color": [255,0,0,255],"size": 8}`

In [38]:
# draw the incidents on the existing map widget
symbol = {"type": "esriSMS","style": "esriSMSCircle","color": [255,0,0,255],"size": 8}
map1.draw(incident_fset, symbol=symbol)
map1
# scroll up to find the map with the incident

MapView(layout=Layout(height='400px', width='100%'))

## Solve for closest hospital
Now, you can solve the closest facility layer you created in the beginning. Syntax: https://developers.arcgis.com/python/api-reference/arcgis.network.toc.html#closestfacilitylayer

Use your incidents feature set for the `incidents` parameter, your hospital feature set for the `facilities` parameter.

Set the `default_target_facility_count` parameter to 1, since we only want to see the routing to the closest hospital

Set the remaining attributes in a way that the impedance is set to travel time and the travel distance is added in kilometers


In [39]:
# solve your closest facillity layer
result = cf_layer.solve_closest_facility(incidents=incident_fset,
                                        facilities=hospitals_fset,
                                        default_target_facility_count=1,
                                        return_facilities=True,
                                        impedance_attribute_name='TravelTime',
                                        accumulate_attribute_names=['Kilometers','TravelTime'])


Exception: {'code': 403, 'subcode': 2, 'message': "User does not have permissions to access 'world/closestfacility.mapserver'.", 'details': []}

Let us inspect the result dictionary. You will use the content of the `routes` dictionary to create the line to be plotted on the map. you can keep the following 3 Code Cells as is, without doing anything

In [ ]:
result.keys()

dict_keys(['requestID', 'routes', 'facilities', 'messages'])

Let us use the `routes` dictionary to construct line features out of the routes to display on the map

In [ ]:
result['routes'].keys()

dict_keys(['fieldAliases', 'geometryType', 'spatialReference', 'fields', 'features'])

In [ ]:
result['routes']['features'][0].keys()

dict_keys(['attributes', 'geometry'])

From here, you need to continue to modify the code

Construct line features out of the routes that are returned. 

The process is the same as before. 

Build a `Feature` for each route (there yould only be one)
Add this to a result list
Build a `FeatureSet` from the list

In [ ]:
# build the list with features
line_feat_list = []
for line_dict in result['routes']['features']:
    f1 = Feature(line_dict['geometry'], line_dict['attributes'])
    line_feat_list.append(f1)


In [ ]:
# create a feature set
routes_fset = FeatureSet(line_feat_list, 
                         geometry_type=result['routes']['geometryType'],
                         spatial_reference= result['routes']['spatialReference'])

Add the routes back to the map using the `draw` method of the map widget once again. Make sure to scroll back up to verify the result

You can use this value for the symbol parameter `{'type': 'esriSLS', 'style': 'esriSLSSolid','color': [0,0,255,100], 'width': 6}`

In [ ]:
# draw the result to the map
symbol = {'type': 'esriSLS', 'style': 'esriSLSSolid','color': [0,0,255,100], 'width': 6}
map1.draw(routes_fset, symbol = symbol)
map1

## Bonus Challenge 
for 2 Extra Points: Label the drawn Line with the distance value from the result object

In [ ]:
for line_feat in line_feat_list:
    map1.draw(line_feat.geometry, popup={"title": "Route", "content": f"Distance: {line_feat.attributes['Total_Kilometers']} km"})
map1